In [ ]:
# -*- coding: utf-8 -*-
import cv2
import numpy as np

#Ipythonで表示用の設定
import matplotlib.pyplot as plt
#%matplotlib notebook

#画像読込
in_img = cv2.imread('/home/haibin/Desktop/plasticStrain0015.png')

# 入力画像をグレースケール変換
gray = cv2.cvtColor(in_img,cv2.COLOR_BGR2GRAY)

#ガウシアンフィルタ処理
gauss = cv2.GaussianBlur(gray,(9,9),.0)

#キャニーエッジ検出
edges = cv2.Canny(gauss,200,500)


#LSD生成
LSD = cv2.createLineSegmentDetector()

#線分検出
lines, width, prec, nfa = LSD.detect(gauss)

# グレースケール変換をBGR変換
color = cv2.cvtColor(gray,cv2.COLOR_GRAY2BGR)

# 直線の描画
for i in range(len(lines)):
    for x1,y1,x2,y2 in lines[i] :
         cv2.line(color,(x1,y1),(x2,y2),(0,0,255),2)

#OpenCVがBGRなのでRGBに変換
disp_in_img  = cv2.cvtColor(color,  cv2.COLOR_BGR2RGB)

#画像表示
plt.figure(figsize=(5,15))
plt.subplot(311)
plt.imshow(gray,cmap='gray')
plt.subplot(312)
plt.imshow(edges,cmap='gray')
plt.subplot(313)
plt.imshow(disp_in_img)
plt.show()

In [ ]:
# Calc line segment length
def lineMagnitude (x1, y1, x2, y2):
    lineMagnitude = math.hypot(x2 - x1,y2 - y1)
    return lineMagnitude

# Calc line center point
def lineCenterPoint (x1, x2):
    centerPoint = ( x1 + x2)/2
    return centerPoint

#Calc minimum distance from a point and a line segment (i.e. consecutive vertices in a polyline).
def DistancePointLine (px, py, x1, y1, x2, y2):
    #http://local.wasp.uwa.edu.au/~pbourke/geometry/pointline/source.vba
    LineMag = lineMagnitude(x1, y1, x2, y2)
 
    if LineMag < 0.00000001:
        DistancePointLine = 9999
        return DistancePointLine
 
    u1 = (((px - x1) * (x2 - x1)) + ((py - y1) * (y2 - y1)))
    u = u1 / (LineMag * LineMag)
 
    if (u < 0.00001) or (u > 1):
        #// closest point does not fall within the line segment, take the shorter distance
        #// to an endpoint
        ix = lineMagnitude(px, py, x1, y1)
        iy = lineMagnitude(px, py, x2, y2)
        if ix >= iy:
            DistancePointLine = iy
        else:
            DistancePointLine = ix
    else:
        # Intersecting point is on the line, use the formula
        ix = x1 + u * (x2 - x1)
        iy = y1 + u * (y2 - y1)
        DistancePointLine = lineMagnitude(px, py, ix, iy)
 
    return DistancePointLine

# Python code to sort a list by keyword of length so as to create 
# another list Use of sorted()
def Sorting(lst):
    lst2 = sorted(lst, key=len)
    return lst2

# detect whether listB is included in listA
def iselement(listA,listB):

    countA = len(listA)
    countBool = np.zeros((count,1))

    for i in range(countA):
        # check whether listB has intersections with listA
        countBool[i] = bool(set(listB) & set(listA[i]))
        # if there is intersections between A and B
        if countBool[i]:
        # insert B into A and form a new A without repeated elements
            listA[i] = np.unique(list(listB)+list(listA[i]))

    return listA, any(countBool)

# refine group with unique function
def regroup(parentA,index):
    result = parentA[index[0]]
    for i in np.arange(1,len(index)):
        result = np.unique(list(result)+list(parentA[index[i]]))
    return result

# select the longest line from same group
def comparelineLength(parentL,lineIndex):
    selectedLine = parentL[int(lineIndex[0])]
    lenthList = []
    lenthIndex = []
    for i in np.arange(len(lineIndex)):
        lenthList.append(parentL[int(lineIndex[i])])
        lenthIndex.append(int(lineIndex[i]))
    index = np.argmax(lenthList)
    return lenthIndex[index]

# check whether one point is within a circle with core(x,y) and radias of rad
def withinDistance(x,y,x1,y1,rad):
    return bool((x-x1)**2+(y-y1)**2<=rad**2)

In [ ]:
def reduceLine(lines,lenscale):
    count = len(lines)
    indexMin = np.zeros((count,2))
    index = np.ones((count,count))
    length = np.zeros((count,1))
    valueMin = np.zeros((count,1))
    for i  in range(count):
        xi1 = lines[i][0][0]
        yi1 = lines[i][0][1]
        xi2 = lines[i][0][2]
        yi2 = lines[i][0][3]
        length[i]=math.hypot(xi2 - xi1,yi2 - yi1)
        for j  in range(count):
            x1 = lines[j][0][0]
            y1 = lines[j][0][1]
            x2 = lines[j][0][2]
            y2 = lines[j][0][3]
            if i == j:
                index[i][j]=1e8
            else:
                index[i][j]= DistancePointLine(lineCenterPoint(xi1,xi2), lineCenterPoint(yi1,yi2), x1, y1, x2, y2)
        indexMin[i][0] = i #min(i,np.argmin(index[i,:]) )
        indexMin[i][1] = np.argmin(index[i,:])#max(i,np.argmin(index[i,:]) )
        valueMin[i] = min (index[i,:])
 
    for i in range(count):
        if valueMin[i]>lenscale:
            indexMin[i][0]=i
            indexMin[i][1]=i
    
    data= indexMin 

    # linking lines sharing common feature of a same nearest line
    grouped = []
    for i in range(len(data)):
        in1 = np.where(((data[i][0] == data) )  )[0]
        in2 = np.where(((data[i][1] == data) )  )[0]
        grouped.append(np.unique(np.concatenate((in1,in2))))

    
    # sort data 
    grouped_sorted = np.array(Sorting(grouped))[::-1]
    uni_group = [grouped_sorted[0]]

    
    for i in range(len(grouped_sorted)):
        for j in range(i+1,len(grouped_sorted)):
            B = grouped_sorted[j]
            uni_group,bl = iselement(uni_group,B)
            if (not bl):
                uni_group.append(B)

    # final refined groups
    uni_group_refine=[]
    for i in range(len(uni_group)):
        uni_group_refine.append(regroup(data,uni_group[i]))
 
    # delet lines of less significance in same group by length
    refined_lines = []
    for i in range(len(uni_group_refine)):
        refined_lines.append(comparelineLength(length,uni_group_refine[i]))
    
    filteredLine = lines[refined_lines]
    return filteredLine

In [ ]:
def linkSegments(lines,rad):
    #input-lines: give segmented lines
    #input-rad: determine searching radius for line end point
    #which genenally is set to be [5 15], depending on resolution
    lineNew = np.copy(lines)
    Nline = len(lines)
    jointPoint = np.zeros((Nline,Nline))
    # seach through all the line end points and find how many neigbour points
    # locating within rad
    for i in range(Nline):
        xi1 = lineNew[i][0][0]
        yi1 = lineNew[i][0][1]
        xi2 = lineNew[i][0][2]
        yi2 = lineNew[i][0][3]
        for j in range(i+1,Nline):
            x1 = lineNew[j][0][0]
            y1 = lineNew[j][0][1]
            x2 = lineNew[j][0][2]
            y2 = lineNew[j][0][3]
            if withinDistance(xi1, yi1, x1,y1,rad):
                # current line end marked 1 with the other 1
                jointPoint[i][j] = 1 
            if withinDistance(xi1, yi1, x2,y2,rad):
                # current line end marked 1 with the other 2
                jointPoint[i][j] = 2
            if withinDistance(xi2, yi2, x1,y1,rad):
                # current line end marked 2 with the other 1
                jointPoint[i][j] = 3
            if withinDistance(xi2, yi2, x2,y2,rad):
                # current line end marked 2 with the other 2
                jointPoint[i][j] = 4

    for i in range(Nline):
        xi1 = lineNew[i][0][0]
        yi1 = lineNew[i][0][1]
        xi2 = lineNew[i][0][2]
        yi2 = lineNew[i][0][3]
        #index(1or2) records neighbour lines whose either end is within either
        #end of current line
        index1 = np.where((jointPoint[i][:]==1) | (jointPoint[i][:]==2))
        index2 = np.where((jointPoint[i][:]==3) | (jointPoint[i][:]==4))
        
        # find lines recorded in index1
        x1 = xi1
        y1 = yi1
        for j in index1[0]:
            if jointPoint[i][j]==1:
                xj1 =  lines[j][0][0]
                yj1 =  lines[j][0][1]
            if jointPoint[i][j]==2:
                xj1 =  lines[j][0][2]
                yj1 =  lines[j][0][3]
            x1 = x1 + xj1
            y1 = y1 + yj1
        #update current line end point1 based on averaged position
        lineNew[i][0][0] = x1/(len(index1[0])+1)
        lineNew[i][0][1] = y1/(len(index1[0])+1)
        
        #update those neighbour lines of end point1 as well
        for j in index1[0]:
            if jointPoint[i][j]==1:
                lineNew[j][0][0] = lineNew[i][0][0] 
                lineNew[j][0][1] = lineNew[i][0][1]
            if jointPoint[i][j]==2:
                lineNew[j][0][2] = lineNew[i][0][0]
                lineNew[j][0][3] = lineNew[i][0][1]

        # find lines recorded in index2
        x2 = xi2
        y2 = yi2
        for j in index2[0]:
            if jointPoint[i][j]==3:
                xj2 =  lines[j][0][0]
                yj2 =  lines[j][0][1]
            if jointPoint[i][j]==4:
                xj2 =  lines[j][0][2]
                yj2 =  lines[j][0][3]
            x2 = x2 + xj2
            y2 = y2 + yj2
        #update current line end point2 based on averaged position
        lineNew[i][0][2] = x2/(len(index2[0])+1)
        lineNew[i][0][3] = y2/(len(index2[0])+1)
        
        #update those neighbour lines of end point2 as well
        for j in index2[0]:
            if jointPoint[i][j]==3:
                lineNew[j][0][0] = lineNew[i][0][2]
                lineNew[j][0][1] = lineNew[i][0][3]
            if jointPoint[i][j]==4:
                lineNew[j][0][2] = lineNew[i][0][2]            
                lineNew[j][0][3] = lineNew[i][0][3]
        # update jointPoint for those neighbours who have been modified
        # in current line, if it is calculated now, when it is counted, 
        # it should not be counted again regaiding current line
        for jj in index1[0]:
            for ii in index1[0]:
                jointPoint[jj][ii]=0
        for jj in index2[0]:
            for ii in index2[0]:  
                jointPoint[jj][ii]=0
                
    return lineNew

In [ ]:
import math
from scipy import stats

shape = lines.shape
count = shape[0]
orient = np.zeros((count,1))
length = np.zeros((count,1))

for i  in range(count):
    x1 = lines[i][0][0]
    y1 = lines[i][0][1]
    x2 = lines[i][0][2]
    y2 = lines[i][0][3]
    length[i]=math.hypot(x2 - x1,y2 - y1)
    orient[i]=math.atan ((y2-y1)/((x2-x1)+1e-18))

# sort lines with orientation first
indexGrt0=np.where((orient>=0) & (length>5))
indexlwr0=np.where((orient<0) & (length>3))

lineGrt0 = lines[indexGrt0[0]]
linelwr0 = lines[indexlwr0[0]]


In [ ]:
LG = reduceLine(lineGrt0,9)
LL = reduceLine(linelwr0,9)
lines = np.concatenate((LG, LL), axis=0)
lineNew = linkSegments(lines,8)

In [ ]:

filteredLine = lineNew
color = cv2.cvtColor(gray,cv2.COLOR_GRAY2BGR)

# 直線の描画
for i in range(len(filteredLine)):
    for x1,y1,x2,y2 in filteredLine[i] :
         cv2.line(color,(x1,y1),(x2,y2),(0,0,255),2)

#OpenCVがBGRなのでRGBに変換
disp_in_img  = cv2.cvtColor(color,  cv2.COLOR_BGR2RGB)

plt.figure(figsize=(5,5))
plt.imshow(disp_in_img)
plt.show()